<a href="https://colab.research.google.com/github/Mavitu56/SLMs/blob/main/SLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch transformers datasets evaluate peft accelerate ipywidgets sentencepiece pandas matplotlib --quiet

In [ ]:
# Importando as bibliotecas necessárias da Hugging Face, como especificado no plano
from datasets import load_dataset
from evaluate import load

def configure_task_and_metrics():
    """
    Define e carrega o dataset e as métricas para a tarefa de avaliação.
    """

    # 1. Definição da Tarefa e Dataset
    # Tarefa: Question Answering (QA) Extrativo
    # Dataset: SQuAD (Stanford Question Answering Dataset)
    # Motivo: Benchmark padrão e disponível no Hugging Face Datasets
    try:
        print("Carregando o dataset SQuAD...")
        # Usaremos apenas o split de validação para a avaliação final, como um conjunto de teste fixo
        squad_dataset_validation = load_dataset("squad", split="validation")
        print("Dataset SQuAD (validação) carregado com sucesso.")
        print(f"Número de exemplos no conjunto de validação: {len(squad_dataset_validation)}")
        print(f"Exemplo de dado: {squad_dataset_validation[0]}")

    except Exception as e:
        print(f"Erro ao carregar o dataset SQuAD: {e}")
        squad_dataset_validation = None

    # 2. Definição das Métricas
    # Métricas: F1-score e Exact Match, padrões para SQuAD.
    # Carregadas usando a biblioteca 'evaluate' da Hugging Face
    try:
        print("\nCarregando as métricas para SQuAD...")
        squad_metric = load("squad")
        print("Métricas para SQuAD carregadas com sucesso.")

    except Exception as e:
        print(f"Erro ao carregar a métrica SQuAD: {e}")
        squad_metric = None

    return squad_dataset_validation, squad_metric

In [ ]:
def select_student_model():
    """
    Seleciona e carrega o SLM base (estudante) e seu tokenizador.
    """
    print("\n--- Ponto 2: Selecionando SLM Base (Estudante) ---")

    # Modelo escolhido: Gemma-2b da Google
    model_id = "google/gemma-2b"
    print(f"Modelo base selecionado: {model_id}")

    try:
        # Carregando o tokenizador
        print(f"Carregando o tokenizador para {model_id}...")
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        print("Tokenizador carregado com sucesso.")

        # Carregando o modelo
        # A utilização de torch.bfloat16 é para otimizar o uso de memória em GPUs compatíveis
        print(f"Carregando o modelo {model_id}...")
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            device_map="auto", # Tenta usar a GPU se disponível
            torch_dtype=torch.bfloat16
        )
        print("Modelo base (estudante) carregado com sucesso.")

    except Exception as e:
        print(f"Erro ao carregar o modelo ou tokenizador: {e}")
        return None, None

    return model, tokenizer

In [ ]:
# FILE: model_loader.py

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

def select_teacher_model():
    """
    Selects and loads the teacher LLM and its tokenizer.
    The teacher model is a larger, more powerful model used for Knowledge Distillation.
    """
    print("\n--- Step 3: Selecting The Teacher Model ---")

    # Model selected: Llama-3-8B-Instruct from Meta
    # Reason: A powerful open-source model to act as the 'teacher'.
    model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
    print(f"Teacher model selected: {model_id}")

    try:
        # Loading the tokenizer
        print(f"Loading tokenizer for {model_id}...")
        # Note: For Llama-3, you might need to request access or use a gated Hugging Face token
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        print("Tokenizer loaded successfully.")

        # Loading the model
        print(f"Loading model {model_id}...")
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            device_map="auto", # Tries to use GPU if available
            torch_dtype=torch.bfloat16
        )
        print("Teacher model loaded successfully.")

    except Exception as e:
        print(f"Error loading the teacher model or tokenizer: {e}")
        print("Please ensure you have access to the model on Hugging Face Hub.")
        return None, None

    return model, tokenizer

In [ ]:
# FILE: data_preparation.py

from datasets import load_dataset

def prepare_datasets(validation_dataset):
    """
    Prepares the datasets needed for training and advanced prompting techniques.

    This includes:
    1. The KD Transfer Set: A subset of the training data for knowledge distillation.
    2. The ICL/CoT Set: A few high-quality examples for in-context learning.
    3. The Evaluation Set: The pre-loaded validation dataset.
    """
    print("\n--- Step 4: Preparing Datasets ---")

    try:
        print("Loading SQuAD training split...")
        squad_train_full = load_dataset("squad", split="train")
        print("SQuAD training split loaded successfully.")

        # For resource-limited environments, we'll create a smaller subset for KD.
        # Shuffling ensures the subset is random and representative.
        kd_transfer_set_size = 10000
        print(f"Creating a random subset of {kd_transfer_set_size} examples for the KD Transfer Set...")
        kd_transfer_set = squad_train_full.shuffle(seed=42).select(range(kd_transfer_set_size))

        # Select a few high-quality examples for In-Context Learning (ICL) and Chain-of-Thought (CoT)
        icl_cot_set_size = 5
        print(f"Selecting {icl_cot_set_size} examples for the ICL/CoT Set...")
        icl_cot_set = squad_train_full.select(range(icl_cot_set_size)) # Taking the first few for simplicity

        datasets = {
            "evaluation": validation_dataset,
            "kd_transfer": kd_transfer_set,
            "icl_cot_examples": icl_cot_set
        }

        print("\nDatasets prepared successfully:")
        print(f" - Evaluation Set size: {len(datasets['evaluation'])}")
        print(f" - KD Transfer Set size: {len(datasets['kd_transfer'])}")
        print(f" - ICL/CoT Examples size: {len(datasets['icl_cot_examples'])}")

        return datasets

    except Exception as e:
        print(f"An error occurred during dataset preparation: {e}")
        return None

# How to use this function in a notebook:
#
# from task_setup import configure_task_and_metrics
# validation_set, metrics = configure_task_and_metrics()
#
# if validation_set:
#     prepared_datasets = prepare_datasets(validation_set)

In [ ]:
import torch

class KDConfig:
    # Device setup
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

    # Model IDs (already decided)
    STUDENT_MODEL_ID = "google/gemma-2b"
    TEACHER_MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"

    # Distillation hyperparameters from the paper
    TEMPERATURE = 2.0  # The 'T' that softens the probabilities
    ALPHA = 0.5        # The weight between KD loss and a potential standard loss

    # Training hyperparameters
    LEARNING_RATE = 5e-5
    NUM_EPOCHS = 3
    BATCH_SIZE = 4     # Adjust based on your GPU memory

In [ ]:
import torch
import torch.nn.functional as F

def compute_distillation_loss(student_logits, teacher_logits, temperature, alpha):
    """
    Computes the Knowledge Distillation loss.
    This loss encourages the student's output distribution to match the teacher's.

    Implements the formula: L_total = alpha * T^2 * L_KD
    Note: The original paper's loss is used here. We omit L_CE for pure distillation.
    The T^2 factor scales the gradient, it's part of the original KD formulation.
    """

    # Calculate the distillation loss (L_KD) based on softened logits
    soft_teacher_probs = F.softmax(teacher_logits / temperature, dim=-1)
    soft_student_log_probs = F.log_softmax(student_logits / temperature, dim=-1)

    # KL Divergence Loss
    distillation_loss = F.kl_div(
        soft_student_log_probs,
        soft_teacher_probs,
        reduction='batchmean' # Averages the loss over the batch
    )

    # The final loss is scaled by T^2 as proposed by Hinton et al.
    # We focus purely on the distillation loss (alpha=1.0) for this base implementation
    # L_total = alpha * (temperature**2) * distillation_loss + (1-alpha) * L_CE
    # For now, let's assume L_CE is not used, so alpha = 1.0

    final_loss = (temperature**2) * distillation_loss

    return final_loss

In [ ]:
from peft import get_peft_model, LoraConfig, TaskType
from torch.optim import AdamW
from tqdm import tqdm # For progress bars

def run_base_kd_training(student_model, teacher_model, tokenizer, kd_dataset, config):
    """
    Runs the base Knowledge Distillation training loop.
    """
    print("\n--- Phase 1, Step 1: Base Knowledge Distillation (Logit Matching) ---")

    # 1. PEFT Setup: Prepare student model for efficient fine-tuning
    lora_config = LoraConfig(
        r=16, # Rank of the update matrices
        lora_alpha=32,
        target_modules=["q_proj", "v_proj"], # Target specific layers
        lora_dropout=0.05,
        bias="none",
        task_type=TaskType.CAUSAL_LM
    )
    peft_student_model = get_peft_model(student_model, lora_config)
    peft_student_model.print_trainable_parameters()

    # 2. Optimizer: Targets only the trainable PEFT parameters
    optimizer = AdamW(peft_student_model.parameters(), lr=config.LEARNING_RATE)

    # 3. Freeze the teacher model's weights
    teacher_model.eval()

    # 4. Training Loop
    peft_student_model.to(config.DEVICE)
    teacher_model.to(config.DEVICE)

    for epoch in range(config.NUM_EPOCHS):
        print(f"\nEpoch {epoch + 1}/{config.NUM_EPOCHS}")
        peft_student_model.train() # Set student to training mode
        total_loss = 0

        for step, batch in enumerate(tqdm(kd_dataset)): # Using a dataloader is recommended
            # Note: A proper dataloader would handle tokenization and batching
            # This is a simplified representation of the logic

            # Dummy tokenized inputs - replace with actual tokenization
            inputs = tokenizer("dummy text", return_tensors="pt").to(config.DEVICE)

            # Get teacher logits (inference mode, no gradients)
            with torch.no_grad():
                teacher_outputs = teacher_model(**inputs)
                teacher_logits = teacher_outputs.logits

            # Get student logits (training mode)
            student_outputs = peft_student_model(**inputs)
            student_logits = student_outputs.logits

            # Calculate loss
            loss = compute_distillation_loss(
                student_logits,
                teacher_logits,
                temperature=config.TEMPERATURE,
                alpha=config.ALPHA
            )

            # Backpropagation to update student's PEFT weights
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            total_loss += loss.item()

    print("Base KD training complete.")

    # 5. Save the resulting model
    # This saves only the trained adapter weights, which is very efficient
    output_dir = "./results/kd_base_model"
    peft_student_model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    print(f"Distilled model (KD Base) saved to {output_dir}")

    return peft_student_model

In [ ]:
# FILE: create_teacher.py
# (Este é um exemplo conceitual de um script de fine-tuning padrão)

from transformers import Trainer, TrainingArguments

def create_specialist_teacher(base_model, tokenizer, train_dataset):
    """
    Fine-tunes a base model on a specific task to create a 'specialist' teacher.
    """
    print("--- Self-Distillation: Creating the Specialist Teacher ---")

    # Define os argumentos de treinamento
    training_args = TrainingArguments(
        output_dir="./results/specialist_teacher",
        num_train_epochs=3,
        per_device_train_batch_size=4,
        learning_rate=2e-5,
        logging_dir='./logs',
        report_to="none" # Desativa integrações como wandb
    )

    # Cria a instância do Trainer
    trainer = Trainer(
        model=base_model,
        args=training_args,
        train_dataset=train_dataset, # O dataset de treino do SQuAD
        tokenizer=tokenizer,
    )

    # Inicia o treinamento
    print("Starting fine-tuning to create the teacher model...")
    trainer.train()
    print("Specialist teacher model created successfully.")

    # Salva o modelo professor
    teacher_output_dir = "./models/gemma_2b_squad_teacher"
    trainer.save_model(teacher_output_dir)
    tokenizer.save_pretrained(teacher_output_dir)

    return base_model

In [ ]:
# FILE: run_self_distillation.py
# (Reutiliza a função de KD Base, apenas muda os modelos de entrada)

from transformers import AutoModelForCausalLM

# Supondo que as funções anteriores já foram definidas:
# from kd_training import run_base_kd_training
# from kd_config import KDConfig

# 1. Carregue o professor "especialista" que você treinou na Fase A
teacher_model_path = "./models/gemma_2b_squad_teacher"
specialist_teacher_model = AutoModelForCausalLM.from_pretrained(teacher_model_path)

# 2. Carregue um modelo estudante "novo" (não treinado)
student_model, tokenizer = select_student_model() # Função que já definimos

# 3. Execute o mesmo loop de treinamento de KD Base
# O código é o mesmo, a diferença é o 'teacher_model' que estamos usando
print("\n--- Running Self-Distillation training ---")
run_base_kd_training(
    student_model=student_model,
    teacher_model=specialist_teacher_model, # A única grande mudança está aqui!
    tokenizer=tokenizer,
    kd_dataset=prepared_datasets['kd_transfer'],
    config=KDConfig()
)

In [ ]:
# FILE: generate_explanations.py

from tqdm import tqdm
from datasets import Dataset

def generate_explanation_dataset(teacher_model, teacher_tokenizer, transfer_dataset, device):
    """
    Uses the teacher model to generate step-by-step explanations and answers.
    """
    print("\n--- Explanation Distillation: Generating new dataset ---")

    # Prompt que instrui o modelo a gerar o raciocínio e a resposta
    prompt_template = """A partir do contexto abaixo, responda à pergunta.
    Explique seu raciocínio passo a passo e termine com a resposta final.
    Use o formato: [RACIOCÍNIO] sua explicação aqui <sep> [RESPOSTA] sua resposta aqui.

    Contexto: {context}
    Pergunta: {question}
    """

    new_data = {"input_text": [], "teacher_output": []}

    teacher_model.to(device)
    teacher_model.eval()

    for example in tqdm(transfer_dataset):
        # Cria o prompt completo
        full_prompt = prompt_template.format(context=example['context'], question=example['question'])
        new_data["input_text"].append(full_prompt)

        # Gera a saída do professor
        inputs = teacher_tokenizer(full_prompt, return_tensors="pt").to(device)
        outputs = teacher_model.generate(**inputs, max_new_tokens=256, pad_token_id=teacher_tokenizer.eos_token_id)
        generated_text = teacher_tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extrai apenas a parte gerada pelo modelo
        teacher_completion = generated_text[len(full_prompt):]
        new_data["teacher_output"].append(teacher_completion)

    # Converte o dicionário em um novo Dataset da Hugging Face
    explanation_dataset = Dataset.from_dict(new_data)
    print("Explanation-augmented dataset created successfully.")

    return explanation_dataset

In [ ]:
# FILE: run_explanation_distillation.py

from transformers import Trainer, TrainingArguments

def run_explanation_kd_training(student_model, tokenizer, explanation_dataset):
    """
    Fine-tunes the student model on the teacher-generated explanations.
    This is a standard causal language modeling task.
    """
    print("--- Explanation Distillation: Training the student ---")

    # O modelo é treinado para gerar a sequência [RACIOCÍNIO] <sep> [RESPOSTA]

    # Função para tokenizar os dados: entrada e saída são concatenadas
    def preprocess_function(examples):
        inputs = examples['input_text']
        targets = examples['teacher_output']
        # Concatenamos para que o modelo aprenda a gerar o alvo a partir da entrada
        model_inputs = [inp + out + tokenizer.eos_token for inp, out in zip(inputs, targets)]
        return tokenizer(model_inputs, truncation=True, max_length=512)

    tokenized_dataset = explanation_dataset.map(preprocess_function, batched=True, remove_columns=explanation_dataset.column_names)

    # Argumentos de treinamento
    training_args = TrainingArguments(
        output_dir="./results/explanation_distilled_model",
        num_train_epochs=3,
        per_device_train_batch_size=2, # Menor batch size pois os textos são mais longos
        learning_rate=2e-5,
        report_to="none",
    )

    # Trainer para a tarefa de modelagem de linguagem causal
    trainer = Trainer(
        model=student_model, # Usamos o PEFT-wrapped model para eficiência
        args=training_args,
        train_dataset=tokenized_dataset,
        tokenizer=tokenizer,
    )

    print("Starting training on explanation data...")
    trainer.train()
    print("Explanation distillation training complete.")

    # Salva o modelo final
    output_dir = "./models/gemma_2b_explanation_distilled"
    trainer.save_model(output_dir)
    tokenizer.save_pretrained(output_dir)

    return student_model

In [ ]:
# FILE: run_evaluation.py

from tqdm import tqdm
import torch

def run_zero_shot_baseline(model, tokenizer, evaluation_dataset, metrics_calculator):
    """
    Executa a avaliação de baseline (Zero-Shot Simples) em um determinado modelo.
    Isto estabelece o desempenho base de cada modelo com uma instrução direta.
    """
    print(f"\n--- Iniciando Avaliação Baseline (Zero-Shot) para o modelo: {model.config._name_or_path} ---")

    # Move o modelo para a GPU, se disponível
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    model.eval()

    # Listas para armazenar as previsões e as respostas de referência
    predictions = []
    references = []

    # Loop sobre o conjunto de avaliação
    for example in tqdm(evaluation_dataset):
        context = example['context']
        question = example['question']
        example_id = example['id']

        # Formato do prompt Zero-Shot
        # Uma instrução direta e simples, conforme o plano
        prompt = f"""Use o contexto abaixo para responder à pergunta.

Contexto: {context}

Pergunta: {question}

Resposta:"""

        # Tokeniza o prompt e gera a resposta
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)

        # Decodifica a resposta e limpa o texto
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        # Extrai apenas a resposta (o texto após o prompt original)
        predicted_answer = generated_text[len(prompt):].strip()

        # Adiciona a previsão no formato esperado pela métrica SQuAD
        predictions.append({'id': example_id, 'prediction_text': predicted_answer})
        # Adiciona a referência no formato esperado pela métrica SQuAD
        references.append({'id': example_id, 'answers': example['answers']})

    # Calcula as métricas finais
    print("Calculando as métricas finais...")
    results = metrics_calculator.compute(predictions=predictions, references=references)

    print(f"\nResultados do Baseline para {model.config._name_or_path}:")
    print(f"  - Exact Match: {results['exact_match']:.2f}%")
    print(f"  - F1-score: {results['f1']:.2f}%")

    return results

# Exemplo de como você chamaria esta função em seu notebook:
#
# # Supondo que 'student_model', 'tokenizer', 'prepared_datasets' e 'squad_metric' já foram carregados
# baseline_results = run_zero_shot_baseline(
#     model=student_model, # Passe o modelo que quer avaliar
#     tokenizer=tokenizer,
#     evaluation_dataset=prepared_datasets['evaluation'],
#     metrics_calculator=squad_metric
# )

In [ ]:
# FILE: run_evaluation.py (continuação)

from tqdm import tqdm
import torch

def run_few_shot_icl(model, tokenizer, evaluation_dataset, icl_examples, num_shots, metrics_calculator):
    """
    Executa a avaliação com In-Context Learning (Few-Shot).
    Testa o desempenho do modelo ao fornecer 'k' exemplos no prompt.
    """
    k = num_shots
    print(f"\n--- Iniciando Avaliação ICL ({k}-Shot) para o modelo: {model.config._name_or_path} ---")

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    model.eval()

    # 1. Construir o prefixo do prompt com os exemplos de ICL
    few_shot_prefix = ""
    for i in range(k):
        example = icl_examples[i]
        # A resposta correta no SQuAD é uma lista, pegamos a primeira
        answer = example['answers']['text'][0]
        few_shot_prefix += f"Contexto: {example['context']}\n\nPergunta: {example['question']}\n\nResposta: {answer}\n\n---\n\n"

    # Listas para previsões e referências
    predictions = []
    references = []

    # 2. Loop sobre o conjunto de avaliação
    for example in tqdm(evaluation_dataset):
        context = example['context']
        question = example['question']
        example_id = example['id']

        # Formato do prompt Few-Shot: prefixo com exemplos + nova pergunta
        prompt = few_shot_prefix + f"Contexto: {context}\n\nPergunta: {question}\n\nResposta:"

        inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True).to(device)
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)

        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        # Extrai apenas a resposta (o texto após o prompt original)
        predicted_answer = generated_text[len(prompt):].strip()

        predictions.append({'id': example_id, 'prediction_text': predicted_answer})
        references.append({'id': example_id, 'answers': example['answers']})

    # 3. Calcula as métricas
    print(f"Calculando as métricas finais para {k}-shot...")
    results = metrics_calculator.compute(predictions=predictions, references=references)

    print(f"\nResultados do ICL ({k}-Shot) para {model.config._name_or_path}:")
    print(f"  - Exact Match: {results['exact_match']:.2f}%")
    print(f"  - F1-score: {results['f1']:.2f}%")

    return results

# Exemplo de como você chamaria esta função em seu notebook para testar k=1, 3 e 5
#
# # Supondo que tudo já foi carregado
# for k_shots in [1, 3, 5]:
#     icl_results = run_few_shot_icl(
#         model=student_model,
#         tokenizer=tokenizer,
#         evaluation_dataset=prepared_datasets['evaluation'],
#         icl_examples=prepared_datasets['icl_cot_examples'], # Nosso conjunto de exemplos de alta qualidade
#         num_shots=k_shots,
#         metrics_calculator=squad_metric
#     )

In [ ]:
# FILE: run_evaluation.py (continuação)
import re

def run_zero_shot_cot(model, tokenizer, evaluation_dataset, metrics_calculator):
    """
    Executa a avaliação com Zero-Shot Chain-of-Thought.
    """
    print(f"\n--- Iniciando Avaliação Zero-Shot CoT para: {model.config._name_or_path} ---")

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    model.eval()

    predictions = []
    references = []

    for example in tqdm(evaluation_dataset):
        context = example['context']
        question = example['question']
        example_id = example['id']

        # Prompt Zero-Shot CoT: Adiciona a instrução para pensar
        prompt = f"""Contexto: {context}

Pergunta: {question}

Pense passo a passo. Responda no formato "A resposta final é: [sua resposta]".
"""
        # A instrução de formato ajuda no pós-processamento.

        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=150, pad_token_id=tokenizer.eos_token_id)

        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        completion = generated_text[len(prompt):].strip()

        # Pós-processamento: Extrair a resposta final
        # Procuramos pelo padrão "A resposta final é:" ou pegamos a última linha
        match = re.search(r'A resposta final é:\s*(.*)', completion, re.IGNORECASE)
        if match:
            predicted_answer = match.group(1).strip()
        else:
            # Se o padrão não for encontrado, usamos uma heurística (ex: última linha)
            predicted_answer = completion.split('\n')[-1].strip()

        predictions.append({'id': example_id, 'prediction_text': predicted_answer})
        references.append({'id': example_id, 'answers': example['answers']})

    print("Calculando as métricas finais para Zero-Shot CoT...")
    results = metrics_calculator.compute(predictions=predictions, references=references)

    print(f"\nResultados do Zero-Shot CoT para {model.config._name_or_path}:")
    print(f"  - Exact Match: {results['exact_match']:.2f}%")
    print(f"  - F1-score: {results['f1']:.2f}%")

    return results

In [ ]:
# FILE: run_evaluation.py (continuação)
import torch
import re
from collections import Counter
from tqdm import tqdm

def run_self_consistency_cot(model, tokenizer, evaluation_dataset, metrics_calculator, num_paths=5):
    """
    Executa a avaliação com Auto-Consistência sobre o Zero-Shot CoT.
    Gera múltiplos caminhos de raciocínio e escolhe a resposta mais comum.
    """
    print(f"\n--- Iniciando Avaliação com Auto-Consistência ({num_paths} caminhos) para: {model.config._name_or_path} ---")

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    model.eval()

    predictions = []
    references = []

    for example in tqdm(evaluation_dataset):
        context = example['context']
        question = example['question']
        example_id = example['id']

        # Prompt é o mesmo do Zero-Shot CoT
        prompt = f"""Contexto: {context}

Pergunta: {question}

Pense passo a passo. Responda no formato "A resposta final é: [sua resposta]".
"""
        inputs = tokenizer(prompt, return_tensors="pt").to(device)

        # 1. Gerar múltiplos caminhos de resposta
        path_answers = []
        for _ in range(num_paths):
            with torch.no_grad():
                # Ativamos a amostragem para gerar saídas diversas
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=150,
                    do_sample=True, # Essencial para a diversidade
                    temperature=0.7, # Controla a aleatoriedade
                    top_k=50,
                    pad_token_id=tokenizer.eos_token_id
                )

            generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
            completion = generated_text[len(prompt):].strip()

            # Pós-processamento para extrair a resposta de cada caminho
            match = re.search(r'A resposta final é:\s*(.*)', completion, re.IGNORECASE)
            if match:
                path_answers.append(match.group(1).strip())

        # 2. Votação majoritária para encontrar a resposta mais consistente
        if path_answers:
            # Usa collections.Counter para contar a frequência de cada resposta
            most_common_answer = Counter(path_answers).most_common(1)[0][0]
            final_prediction = most_common_answer
        else:
            final_prediction = "" # Se nenhum caminho produzir uma resposta válida

        predictions.append({'id': example_id, 'prediction_text': final_prediction})
        references.append({'id': example_id, 'answers': example['answers']})

    print("Calculando as métricas finais para Auto-Consistência...")
    results = metrics_calculator.compute(predictions=predictions, references=references)

    print(f"\nResultados da Auto-Consistência para {model.config._name_or_path}:")
    print(f"  - Exact Match: {results['exact_match']:.2f}%")
    print(f"  - F1-score: {results['f1']:.2f}%")

    return results

# Exemplo de como chamar a função em seu notebook:
#
# self_consistency_results = run_self_consistency_cot(
#     model=student_model,
#     tokenizer=tokenizer,
#     evaluation_dataset=prepared_datasets['evaluation'],
#     metrics_calculator=squad_metric,
#     num_paths=5 # Define quantos caminhos de raciocínio gerar
# )

In [ ]:
# -*- coding: utf-8 -*-

"""
Fase 3: Avaliação e Comparação Científica Sistemática
Este script orquestra a execução de todos os experimentos, coleta as métricas,
analisa os resultados e gera um relatório final.
"""

import pandas as pd
import matplotlib.pyplot as plt
import time

# --- Simulação das Funções e Modelos das Fases Anteriores ---
# Em um cenário real, você importaria estas funções e carregaria os modelos.
# Para este exemplo, vamos simular suas saídas para focar na lógica da Fase 3.

def load_model_and_tokenizer(model_name):
    """Função de simulação para carregar um modelo."""
    print(f"Carregando modelo simulado: {model_name}...")
    # Retorna um objeto 'model' e 'tokenizer' simulados
    return {"name": model_name}, {"name": f"{model_name}-tokenizer"}

def run_zero_shot_baseline(model, tokenizer, dataset, metrics):
    """Simula a execução do baseline."""
    # Retorna resultados simulados para cada modelo
    scores = {"SLM Base": 55.3, "KD Base": 62.1, "KD Auto-Destilado": 59.8, "KD com Explicações": 65.5}
    score = scores.get(model['name'], 50.0) + (hash(model['name']) % 10) # Adiciona variação
    return {'f1': score, 'exact_match': score - 10}

def run_few_shot_icl(model, tokenizer, dataset, icl_examples, num_shots, metrics):
    """Simula a execução do ICL."""
    base_score = run_zero_shot_baseline(model, tokenizer, dataset, metrics)['f1']
    return {'f1': base_score + 5 + num_shots, 'exact_match': base_score - 5 + num_shots}

def run_zero_shot_cot(model, tokenizer, dataset, metrics):
    """Simula a execução do CoT."""
    base_score = run_zero_shot_baseline(model, tokenizer, dataset, metrics)['f1']
    return {'f1': base_score + 8, 'exact_match': base_score - 2}

def run_self_consistency_cot(model, tokenizer, dataset, metrics, num_paths):
    """Simula a execução da Auto-Consistência."""
    base_score = run_zero_shot_cot(model, tokenizer, dataset, metrics)['f1']
    return {'f1': base_score + 4, 'exact_match': base_score + 2}

# --- Implementação da Fase 3 ---

def step_1_create_experiment_matrix():
    """
    Passo 1: Cria a Matriz de Experimentos.
    Define os modelos a serem testados e as estratégias de prompting a serem aplicadas.
    """
    print("## Passo 1: Definindo a Matriz de Experimentos ##\n")

    # Linhas da matriz: Nossas versões de modelo
    model_versions = {
        "SLM Base": "google/gemma-2b",
        "KD Base": "./models/kd_base_model",
        "KD Auto-Destilado": "./models/kd_self_distilled_model",
        "KD com Explicações": "./models/explanation_distilled_model"
    }

    # Colunas da matriz: Nossas estratégias de prompting
    prompting_strategies = {
        "Zero-Shot Simples": run_zero_shot_baseline,
        "ICL (k=3)": lambda m, t, d, met: run_few_shot_icl(m, t, d, None, 3, met),
        "Zero-Shot CoT": run_zero_shot_cot,
        "Auto-Consistência (n=5)": lambda m, t, d, met: run_self_consistency_cot(m, t, d, met, 5)
    }

    # Cria um DataFrame pandas vazio para armazenar os resultados
    results_df = pd.DataFrame(index=model_versions.keys(), columns=prompting_strategies.keys())

    return model_versions, prompting_strategies, results_df


def step_2_and_3_run_experiments_and_collect_metrics(model_versions, prompting_strategies, results_df):
    """
    Passos 2 & 3: Executa todos os experimentos e coleta as métricas.
    Preenche a matriz com os resultados de F1-score / Exact Match.
    """
    print("## Passos 2 & 3: Executando Experimentos e Coletando Métricas ##\n")

    # Simulação de datasets e métricas
    dummy_dataset = {}
    dummy_metrics = {}

    for model_alias, model_path in model_versions.items():
        model, tokenizer = load_model_and_tokenizer(model_alias) # Nome do modelo usado como alias

        for strategy_name, strategy_func in prompting_strategies.items():
            print(f"Executando '{strategy_name}' para o modelo '{model_alias}'...")
            start_time = time.time()

            # Executa a função de avaliação correspondente
            results = strategy_func(model, tokenizer, dummy_dataset, dummy_metrics)

            # Coleta métricas de desempenho e eficiência (opcional)
            f1 = results['f1']
            em = results['exact_match']
            latency = time.time() - start_time

            # Preenche a matriz
            results_df.loc[model_alias, strategy_name] = f"{f1:.1f} / {em:.1f}"
            # Poderíamos adicionar a latência a outra matriz se quiséssemos
            print(f"Resultado (F1/EM): {f1:.1f}/{em:.1f} | Latência: {latency:.2f}s")

    return results_df


def step_4_analyze_results(results_df):
    """
    Passo 4: Analisa os resultados para responder às perguntas científicas.
    """
    print("\n## Passo 4: Análise dos Resultados ##\n")

    # Converte os resultados para um tipo numérico para análise (usando F1 como principal)
    numeric_df = results_df.applymap(lambda x: float(x.split('/')[0]))

    analysis_summary = []

    # Pergunta: Qual técnica de KD foi melhor?
    avg_kd_performance = numeric_df.drop("SLM Base").mean(axis=1)
    best_kd_model = avg_kd_performance.idxmax()
    analysis_summary.append(f"Melhor Técnica de KD: '{best_kd_model}' com uma média de F1 de {avg_kd_performance.max():.2f}.")

    # Pergunta: Qual estratégia de prompting foi melhor?
    avg_prompt_performance = numeric_df.mean(axis=0)
    best_prompt_strategy = avg_prompt_performance.idxmax()
    analysis_summary.append(f"Melhor Estratégia de Prompting: '{best_prompt_strategy}' com uma média de F1 de {avg_prompt_performance.max():.2f}.")

    # Pergunta: Houve sinergia?
    best_combo = numeric_df.stack().idxmax()
    analysis_summary.append(f"Melhor Sinergia (Combinação): O modelo '{best_combo[0]}' com a estratégia '{best_combo[1]}' atingiu o maior F1 de {numeric_df.max().max():.2f}.")

    print("\nResumo da Análise:")
    for point in analysis_summary:
        print(f"- {point}")

    return analysis_summary, numeric_df


def step_5_document_and_report(results_df, numeric_df, analysis_summary):
    """
    Passo 5: Documenta e relata os resultados.
    Cria uma visualização e um resumo em texto.
    """
    print("\n## Passo 5: Documentação e Relatório Final ##\n")

    report = f"""
# Relatório de Aprimoramento e Comparação de SLMs

## 1. Resumo da Análise
"""
    for point in analysis_summary:
        report += f"- {point}\n"

    report += "\n## 2. Matriz de Resultados (F1-Score / Exact Match)\n"
    report += results_df.to_markdown()

    print("Gerando relatório em formato Markdown...")
    print("-------------------------------------------")
    print(report)
    print("-------------------------------------------")

    # Gera um gráfico de barras para a performance do baseline
    fig, ax = plt.subplots()
    numeric_df['Zero-Shot Simples'].plot(kind='bar', ax=ax, title='Performance Baseline (F1-Score)', legend=False)
    ax.set_ylabel("F1-Score")
    ax.set_xlabel("Versão do Modelo")
    ax.tick_params(axis='x', rotation=45)
    plt.tight_layout()

    # Salva o gráfico
    output_filename = "baseline_performance_comparison.png"
    plt.savefig(output_filename)
    print(f"\nGráfico de comparação salvo em '{output_filename}'.")


if __name__ == '__main__':
    # Orquestra a execução de toda a Fase 3
    model_defs, strategy_defs, df = step_1_create_experiment_matrix()
    completed_df = step_2_and_3_run_experiments_and_collect_metrics(model_defs, strategy_defs, df)
    summary, numeric_results_df = step_4_analyze_results(completed_df)
    step_5_document_and_report(completed_df, numeric_results_df, summary)

# Workflow Completo: Aprimoramento e Comparação de SLMs

Este notebook é a versão final, completa e funcional do plano para aprimorar e comparar um Small Language Model (SLM) usando Destilação de Conhecimento e Engenharia de Prompt.

### 1. Instalação e Importações Globais

In [ ]:
!pip install torch transformers "datasets==2.19.0" evaluate peft accelerate ipywidgets sentencepiece pandas matplotlib --quiet

In [ ]:
import torch
import re
import pandas as pd
import matplotlib.pyplot as plt
import time
from collections import Counter
from tqdm.notebook import tqdm
from datasets import load_dataset, Dataset
from evaluate import load
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType
from torch.optim import AdamW
import torch.nn.functional as F

### 2. Funções da Fase 0: Configuração do Ambiente

In [ ]:
def configure_task_and_metrics():
    """Carrega o dataset e as métricas para a tarefa de avaliação."""
    print("--- Step 0.1: Configuring Task (SQuAD) and Metrics ---")
    validation_dataset = load_dataset("squad", split="validation")
    squad_metric = load("squad")
    print("Dataset e métricas carregados.")
    return validation_dataset, squad_metric

def load_model(model_id):
    """Carrega um modelo e seu tokenizador a partir de um ID."""
    print(f"\n--- Loading Model: {model_id} ---")
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        # Adiciona um pad token se não existir, comum para modelos de geração
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
        model = AutoModelForCausalLM.from_pretrained(
            model_id, device_map="auto", torch_dtype=torch.bfloat16
        )
        print("Modelo carregado com sucesso.")
        return model, tokenizer
    except Exception as e:
        print(f"Erro ao carregar o modelo: {e}")
        return None, None

def prepare_datasets(validation_dataset):
    """Prepara todos os subconjuntos de dados necessários."""
    print("\n--- Step 0.4: Preparing Datasets ---")
    squad_train_full = load_dataset("squad", split="train")
    # Usamos um subset menor para o KD Transfer para agilizar o treinamento
    kd_transfer_set = squad_train_full.shuffle(seed=42).select(range(10000))
    icl_cot_set = squad_train_full.select(range(5))
    datasets = {
        "evaluation": validation_dataset,
        "kd_transfer": kd_transfer_set,
        "icl_cot_examples": icl_cot_set
    }
    print("Datasets preparados.")
    return datasets

### 3. Funções da Fase 1: Destilação de Conhecimento (KD)

In [ ]:
class KDConfig:
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    TEMPERATURE = 2.0
    LEARNING_RATE = 5e-5
    NUM_EPOCHS = 1 # Reduzido para uma execução mais rápida de exemplo. Aumente para 3 em um treino real.

# --- Técnica 1.1: KD Base (Logit Matching) ---
def compute_distillation_loss(student_logits, teacher_logits, temperature):
    """Calcula a perda de Destilação de Conhecimento (KL Divergence)."""
    soft_teacher_probs = F.softmax(teacher_logits / temperature, dim=-1)
    soft_student_log_probs = F.log_softmax(student_logits / temperature, dim=-1)
    distillation_loss = F.kl_div(soft_student_log_probs, soft_teacher_probs, reduction='batchmean')
    return (temperature**2) * distillation_loss

def run_base_kd_training(student_model, teacher_model, student_tokenizer, teacher_tokenizer, kd_dataset, config, output_dir):
    """Executa o loop de treinamento para a Destilação de Conhecimento base."""
    print(f"\n--- Running Base Knowledge Distillation -> saving to {output_dir} ---")
    lora_config = LoraConfig(r=16, lora_alpha=32, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none", task_type=TaskType.CAUSAL_LM)
    peft_student_model = get_peft_model(student_model, lora_config)
    optimizer = AdamW(peft_student_model.parameters(), lr=config.LEARNING_RATE)
    teacher_model.eval()

    for epoch in range(config.NUM_EPOCHS):
        print(f"\nEpoch {epoch + 1}/{config.NUM_EPOCHS}")
        peft_student_model.train()
        for batch in tqdm(kd_dataset.shuffle(seed=epoch).select(range(200)), desc=f"Epoch {epoch+1}"):
            prompt = f"Contexto: {batch['context']}\n\nPergunta: {batch['question']}"

            # Tokeniza para o PROFESSOR com o tokenizador do PROFESSOR
            teacher_inputs = teacher_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(config.DEVICE)

            # Tokeniza para o ESTUDANTE com o tokenizador do ESTUDANTE
            student_inputs = student_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(config.DEVICE)

            # Gera os logits de cada modelo com seus respectivos inputs
            with torch.no_grad():
                teacher_logits = teacher_model(**teacher_inputs).logits

            student_logits = peft_student_model(**student_inputs).logits

            # A perda é calculada entre os logits, que devem ter o mesmo tamanho de vocabulário
            # Se os vocabulários forem diferentes, a destilação de logits não é diretamente aplicável
            # e a "Destilação de Explicações" se torna a abordagem correta.
            # Adicionando uma verificação para evitar o erro:
            if student_logits.shape[-1] != teacher_logits.shape[-1]:
                raise ValueError(
                    f"Os tamanhos dos vocabulários do estudante ({student_logits.shape[-1]}) e do professor ({teacher_logits.shape[-1]}) são diferentes. "
                    "A destilação de logits direta não é possível. Use a Destilação de Explicações."
                )

            loss = compute_distillation_loss(student_logits, teacher_logits, config.TEMPERATURE)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
    peft_student_model.save_pretrained(output_dir)
    student_tokenizer.save_pretrained(output_dir)
    print(f"Modelo destilado salvo em {output_dir}")

# --- Técnica 1.2: Destilação de Explicações ---
def generate_explanation_dataset(teacher_model, tokenizer, transfer_dataset):
    """Usa o modelo professor para gerar um dataset com raciocínios."""
    print("\n--- Generating Explanation-Augmented Dataset ---")
    prompt_template = "A partir do contexto, responda à pergunta. Explique seu raciocínio passo a passo e termine com a resposta final. Formato: [RACIOCÍNIO] ... <sep> [RESPOSTA] ...\n\nContexto: {context}\nPergunta: {question}"
    new_data = {"text": []}
    for example in tqdm(transfer_dataset.select(range(200)), desc="Generating Explanations"):
        full_prompt = prompt_template.format(context=example['context'], question=example['question'])
        inputs = tokenizer(full_prompt, return_tensors="pt").to(KDConfig.DEVICE)
        outputs = teacher_model.generate(**inputs, max_new_tokens=256, pad_token_id=tokenizer.eos_token_id)
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        new_data["text"].append(generated_text)
    return Dataset.from_dict(new_data)

def run_explanation_kd_training(student_model, tokenizer, explanation_dataset, config, output_dir):
    """Treina o estudante para gerar as explicações do professor."""
    print(f"\n--- Running Explanation Distillation -> saving to {output_dir} ---")
    lora_config = LoraConfig(r=16, lora_alpha=32, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none", task_type=TaskType.CAUSAL_LM)
    peft_student_model = get_peft_model(student_model, lora_config)
    training_args = TrainingArguments(output_dir="./results/explanation_training", num_train_epochs=config.NUM_EPOCHS, learning_rate=config.LEARNING_RATE, report_to="none")
    trainer = Trainer(model=peft_student_model, args=training_args, train_dataset=explanation_dataset, tokenizer=tokenizer, data_collator=lambda data: {'input_ids': torch.stack([f['input_ids'] for f in data]), 'attention_mask': torch.stack([f['attention_mask'] for f in data]), 'labels': torch.stack([f['input_ids'] for f in data])})
    trainer.train()
    peft_student_model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    print(f"Modelo de explicação destilada salvo em {output_dir}")

### 4. Funções da Fase 2: Avaliação com Engenharia de Prompt

In [ ]:
def run_zero_shot_baseline(model, tokenizer, evaluation_dataset, metrics_calculator):
    """Executa a avaliação de baseline (Zero-Shot Simples)."""
    prompt = "Contexto: {context}\n\nPergunta: {question}\n\nResposta:"
    return run_generic_evaluation(model, tokenizer, evaluation_dataset, metrics_calculator, prompt, "Zero-Shot Baseline")

def run_few_shot_icl(model, tokenizer, evaluation_dataset, icl_examples, num_shots, metrics_calculator):
    """Executa a avaliação com In-Context Learning (Few-Shot)."""
    prefix = "".join([f"Contexto: {ex['context']}\nPergunta: {ex['question']}\nResposta: {ex['answers']['text'][0]}\n\n---\n\n" for ex in icl_examples.select(range(num_shots))])
    prompt = prefix + "Contexto: {context}\n\nPergunta: {question}\n\nResposta:"
    return run_generic_evaluation(model, tokenizer, evaluation_dataset, metrics_calculator, prompt, f"{num_shots}-Shot ICL")

def run_zero_shot_cot(model, tokenizer, evaluation_dataset, metrics_calculator):
    """Executa a avaliação com Zero-Shot Chain-of-Thought."""
    prompt = "Contexto: {context}\n\nPergunta: {question}\n\nPense passo a passo. A resposta final é:"
    return run_generic_evaluation(model, tokenizer, evaluation_dataset, metrics_calculator, prompt, "Zero-Shot CoT")

def run_self_consistency_cot(model, tokenizer, evaluation_dataset, metrics_calculator, num_paths=5):
    """Executa a avaliação com Auto-Consistência sobre o Zero-Shot CoT."""
    print(f"\n--- Running Self-Consistency ({num_paths} paths) ---")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.eval()
    prompt_template = "Contexto: {context}\n\nPergunta: {question}\n\nPense passo a passo. A resposta final é:"

    predictions, references = [], []
    for example in tqdm(evaluation_dataset.select(range(20)), desc="Self-Consistency Eval"):
        prompt = prompt_template.format(context=example['context'], question=example['question'])
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        path_answers = []
        for _ in range(num_paths):
            with torch.no_grad():
                outputs = model.generate(**inputs, max_new_tokens=150, do_sample=True, temperature=0.7, top_k=50, pad_token_id=tokenizer.eos_token_id)
            completion = tokenizer.decode(outputs[0], skip_special_tokens=True)[len(prompt):].strip()
            match = re.search(r'A resposta final é:\s*(.*)', completion, re.IGNORECASE)
            if match: path_answers.append(match.group(1).strip())

        final_prediction = Counter(path_answers).most_common(1)[0][0] if path_answers else ""
        predictions.append({'id': example['id'], 'prediction_text': final_prediction})
        references.append({'id': example['id'], 'answers': example['answers']})
    return metrics_calculator.compute(predictions=predictions, references=references)

def run_generic_evaluation(model, tokenizer, eval_dataset, metrics_calculator, prompt_template, strategy_name, eval_subset_size=50):
    """Função genérica que executa a avaliação para a maioria das estratégias de prompt."""
    print(f"\n--- Running Evaluation: {strategy_name} ---")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.eval()
    predictions, references = [], []

    for example in tqdm(eval_dataset.select(range(eval_subset_size)), desc=f"Evaluating {strategy_name}"):
        prompt = prompt_template.format(context=example['context'], question=example['question'])
        inputs = tokenizer(prompt, return_tensors="pt", max_length=1536, truncation=True).to(device)
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=60, pad_token_id=tokenizer.eos_token_id)

        full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        predicted_answer = full_text[len(prompt):].strip()

        if "A resposta final é:" in prompt:
            match = re.search(r'A resposta final é:\s*(.*)', predicted_answer, re.IGNORECASE)
            predicted_answer = match.group(1).strip() if match else predicted_answer.split('\n')[-1].strip()

        predictions.append({'id': example['id'], 'prediction_text': predicted_answer})
        references.append({'id': example['id'], 'answers': example['answers']})

    return metrics_calculator.compute(predictions=predictions, references=references)

### 5. Execução do Pipeline e Geração de Relatório (Fase 3)

In [ ]:
from peft import PeftModel

def run_real_workflow(run_training=False, eval_subset_size=50):
    """
    Orquestra o pipeline completo e REAL: setup, treinamento (opcional), avaliação e relatório.

    Args:
        run_training (bool): Se True, executa a demorada fase de treinamento dos modelos KD.
                             Execute como True uma vez para gerar os modelos. Depois, use False.
        eval_subset_size (int): Número de exemplos do dataset de validação para usar na avaliação.
                                Aumente para uma avaliação mais robusta.
    """
    # --- FASE 0: SETUP ---
    print("--- FASE 0: CONFIGURANDO AMBIENTE E DADOS ---")
    validation_data, metrics_calc = configure_task_and_metrics()
    prepared_data = prepare_datasets(validation_data)
    config = KDConfig()

    # --- FASE 1: TREINAMENTO (Opcional) ---
    if run_training:
        print("\n--- FASE 1: EXECUTANDO TREINAMENTO DOS MODELOS KD ---")
        # Carrega os modelos base necessários para todos os treinamentos
        student_model_base, student_tokenizer = load_model("google/gemma-2b")
        teacher_model_ext, teacher_tokenizer = load_model("mistralai/Mistral-7B-Instruct-v0.2")

        # 1. Treina o modelo de KD Base
        run_base_kd_training(
            student_model=student_model_base, teacher_model=teacher_model_ext, tokenizer=tokenizer,
            kd_dataset=prepared_data['kd_transfer'], config=config, output_dir="./results/kd_base_model"
        )

        # 2. Treina o modelo de Auto-Destilação (requer um professor "especialista")
        print("\n--- Treinando professor para Auto-Destilação ---")
        # A forma mais simples de criar um professor especialista é com um fine-tuning padrão.
        # Aqui, vamos simular isso treinando o próprio estudante com KD por uma época.
        specialist_teacher = run_base_kd_training(
            student_model=student_model_base, teacher_model=teacher_model_ext, tokenizer=tokenizer,
            kd_dataset=prepared_data['kd_transfer'], config=config, output_dir="./results/specialist_teacher_for_self_distillation"
        )
        print("\n--- Executando Auto-Destilação ---")
        run_base_kd_training(
            student_model=student_model_base, teacher_model=specialist_teacher, tokenizer=tokenizer,
            kd_dataset=prepared_data['kd_transfer'], config=config, output_dir="./results/kd_self_distilled_model"
        )

        # 3. Treina o modelo de Destilação de Explicações
        explanation_dataset = generate_explanation_dataset(teacher_model_ext, tokenizer, prepared_data['kd_transfer'])
        run_explanation_kd_training(
            student_model=student_model_base, tokenizer=tokenizer, explanation_dataset=explanation_dataset,
            config=config, output_dir="./results/explanation_distilled_model"
        )
        print("\n--- FASE DE TREINAMENTO CONCLUÍDA ---")

    # --- FASE 2 & 3: AVALIAÇÃO E ANÁLISE ---
    print("\n--- FASES 2 & 3: EXECUTANDO AVALIAÇÃO COMPLETA E GERANDO RELATÓRIO ---")
    model_definitions = {
        "SLM Base": "google/gemma-2b",
        "KD Base": "./results/kd_base_model",
        "KD Auto-Destilado": "./results/kd_self_distilled_model",
        "KD com Explicações": "./results/explanation_distilled_model"
    }
    prompting_strategies = {
        "Zero-Shot Simples": run_zero_shot_baseline,
        "ICL (k=3)": lambda m, t, d, met: run_few_shot_icl(m, t, d, prepared_data['icl_cot_examples'], 3, met),
        "Zero-Shot CoT": run_zero_shot_cot,
        "Auto-Consistência (n=5)": lambda m, t, d, met: run_self_consistency_cot(m, t, d, met, 5)
    }
    results_df = pd.DataFrame(index=model_definitions.keys(), columns=prompting_strategies.keys())

    # Carrega o modelo base uma vez para aplicar os adaptadores PEFT
    base_model_for_eval, tokenizer = load_model("google/gemma-2b")

    for alias, path in model_definitions.items():
        print(f"\n>>>> AVALIANDO MODELO: {alias} <<<<")
        if alias == "SLM Base":
            model = base_model_for_eval
        else:
            try:
                # Carrega o modelo base com o adaptador LoRA treinado
                model = PeftModel.from_pretrained(base_model_for_eval, path)
                model = model.merge_and_unload() # Opcional: mescla os pesos para acelerar a inferência
            except Exception as e:
                print(f"Não foi possível carregar o modelo treinado de '{path}'. Pulando. Erro: {e}")
                continue

        for strat_name, strat_func in prompting_strategies.items():
            # A função de avaliação genérica agora é chamada por suas wrappers específicas
            results = strat_func(model, tokenizer, prepared_data['evaluation'], metrics_calc)
            results_df.loc[alias, strat_name] = f"{results['f1']:.1f} / {results['exact_match']:.1f}"

    # --- RELATÓRIO FINAL ---
    print("\n\n## Relatório Final de Análise ##")
    numeric_df = results_df.applymap(lambda x: float(x.split('/')[0]) if isinstance(x, str) else 0)

    print("\n--- Matriz de Resultados (F1-Score / Exact Match) ---")
    print(results_df.to_markdown())

    print("\n--- Conclusões da Análise ---")
    avg_kd_performance = numeric_df.drop("SLM Base").mean(axis=1)
    print(f"- Melhor Técnica de KD (Média F1): '{avg_kd_performance.idxmax()}' ({avg_kd_performance.max():.2f})")
    avg_prompt_performance = numeric_df.mean(axis=0)
    print(f"- Melhor Estratégia de Prompting (Média F1): '{avg_prompt_performance.idxmax()}' ({avg_prompt_performance.max():.2f})")
    best_combo = numeric_df.stack().idxmax()
    print(f"- Melhor Sinergia (Modelo + Prompt): '{best_combo[0]}' + '{best_combo[1]}' com F1 de {numeric_df.max().max():.2f}")

    print("\nGerando gráfico de comparação...")
    fig, ax = plt.subplots(figsize=(12, 7))
    numeric_df.plot(kind='bar', ax=ax, title='Comparação de Performance (F1-Score) entre Modelos e Estratégias')
    ax.set_ylabel("F1-Score")
    ax.set_xlabel("Versão do Modelo")
    ax.tick_params(axis='x', rotation=20)
    plt.legend(title='Estratégia de Prompt', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.savefig("real_run_full_performance_comparison.png")
    print("Gráfico salvo em 'real_run_full_performance_comparison.png'.")

### 6. Execução do Pipeline (Modo Simulado)

A célula abaixo executa o pipeline em **modo de simulação**. Ela não treina nem avalia os modelos de verdade, mas gera um relatório com dados de exemplo para demonstrar a estrutura de análise. Para uma execução real, você deve primeiro executar as células de treinamento da Fase 1 e depois rodar o pipeline com `simulation_mode=False`.

In [ ]:
# prompt: login hugging face

from huggingface_hub import notebook_login
notebook_login()

In [ ]:
run_full_pipeline(simulation_mode=True)

NameError: name 'run_full_pipeline' is not defined

In [ ]:
from peft import PeftModel

def run_real_workflow(run_training=False, eval_subset_size=50):
    """
    Orquestra o pipeline completo e REAL: setup, treinamento (opcional), avaliação e relatório.

    Args:
        run_training (bool): Se True, executa a demorada fase de treinamento dos modelos KD.
                             Execute como True uma vez para gerar os modelos. Depois, use False.
        eval_subset_size (int): Número de exemplos do dataset de validação para usar na avaliação.
                                Aumente para uma avaliação mais robusta.
    """
    # --- FASE 0: SETUP ---
    print("--- FASE 0: CONFIGURANDO AMBIENTE E DADOS ---")
    validation_data, metrics_calc = configure_task_and_metrics()
    prepared_data = prepare_datasets(validation_data)
    config = KDConfig()

    # --- FASE 1: TREINAMENTO (Opcional) ---
    if run_training:
        print("\n--- FASE 1: EXECUTANDO TREINAMENTO DOS MODELOS KD ---")
        # Carrega os modelos base necessários para todos os treinamentos
        student_model_base, student_tokenizer = load_model("google/gemma-2b")
        teacher_model_ext, teacher_tokenizer = load_model("mistralai/Mistral-7B-Instruct-v0.2")
        print(teacher_model_ext)

        # 1. Treina o modelo de KD Base
        run_base_kd_training(
            student_model=student_model_base, teacher_model=teacher_model_ext, student_tokenizer=student_tokenizer, teacher_tokenizer=teacher_tokenizer,
            kd_dataset=prepared_data['kd_transfer'], config=config, output_dir="./results/kd_base_model"
        )

        # 2. Treina o modelo de Auto-Destilação (requer um professor "especialista")
        print("\n--- Treinando professor para Auto-Destilação ---")
        # A forma mais simples de criar um professor especialista é com um fine-tuning padrão.
        # Aqui, vamos simular isso treinando o próprio estudante com KD por uma época.
        specialist_teacher = run_base_kd_training(
            student_model=student_model_base, teacher_model=teacher_model_ext, student_tokenizer=student_tokenizer, teacher_tokenizer=teacher_tokenizer,
            kd_dataset=prepared_data['kd_transfer'], config=config, output_dir="./results/specialist_teacher_for_self_distillation"
        )
        print("\n--- Executando Auto-Destilação ---")
        run_base_kd_training(
            student_model=student_model_base, teacher_model=specialist_teacher, student_tokenizer=student_tokenizer, teacher_tokenizer=teacher_tokenizer,
            kd_dataset=prepared_data['kd_transfer'], config=config, output_dir="./results/kd_self_distilled_model"
        )

        # 3. Treina o modelo de Destilação de Explicações
        explanation_dataset = generate_explanation_dataset(teacher_model_ext, teacher_tokenizer, prepared_data['kd_transfer'])
        run_explanation_kd_training(
            student_model=student_model_base, student_tokenizer=student_tokenizer, teacher_tokenizer=teacher_tokenizer, explanation_dataset=explanation_dataset,
            config=config, output_dir="./results/explanation_distilled_model"
        )
        print("\n--- FASE DE TREINAMENTO CONCLUÍDA ---")

    # --- FASE 2 & 3: AVALIAÇÃO E ANÁLISE ---
    print("\n--- FASES 2 & 3: EXECUTANDO AVALIAÇÃO COMPLETA E GERANDO RELATÓRIO ---")
    model_definitions = {
        "SLM Base": "google/gemma-2b",
        "KD Base": "./results/kd_base_model",
        "KD Auto-Destilado": "./results/kd_self_distilled_model",
        "KD com Explicações": "./results/explanation_distilled_model"
    }
    prompting_strategies = {
        "Zero-Shot Simples": run_zero_shot_baseline,
        "ICL (k=3)": lambda m, t, d, met: run_few_shot_icl(m, t, d, prepared_data['icl_cot_examples'], 3, met),
        "Zero-Shot CoT": run_zero_shot_cot,
        "Auto-Consistência (n=5)": lambda m, t, d, met: run_self_consistency_cot(m, t, d, met, 5)
    }
    results_df = pd.DataFrame(index=model_definitions.keys(), columns=prompting_strategies.keys())

    # Carrega o modelo base uma vez para aplicar os adaptadores PEFT
    base_model_for_eval, tokenizer = load_model("google/gemma-2b")

    for alias, path in model_definitions.items():
        print(f"\n>>>> AVALIANDO MODELO: {alias} <<<<")
        if alias == "SLM Base":
            model = base_model_for_eval
        else:
            try:
                # Carrega o modelo base com o adaptador LoRA treinado
                model = PeftModel.from_pretrained(base_model_for_eval, path)
                model = model.merge_and_unload() # Opcional: mescla os pesos para acelerar a inferência
            except Exception as e:
                print(f"Não foi possível carregar o modelo treinado de '{path}'. Pulando. Erro: {e}")
                continue

        for strat_name, strat_func in prompting_strategies.items():
            # A função de avaliação genérica agora é chamada por suas wrappers específicas
            results = strat_func(model, tokenizer, prepared_data['evaluation'], metrics_calc)
            results_df.loc[alias, strat_name] = f"{results['f1']:.1f} / {results['exact_match']:.1f}"

    # --- RELATÓRIO FINAL ---
    print("\n\n## Relatório Final de Análise ##")
    numeric_df = results_df.applymap(lambda x: float(x.split('/')[0]) if isinstance(x, str) else 0)

    print("\n--- Matriz de Resultados (F1-Score / Exact Match) ---")
    print(results_df.to_markdown())

    print("\n--- Conclusões da Análise ---")
    avg_kd_performance = numeric_df.drop("SLM Base").mean(axis=1)
    print(f"- Melhor Técnica de KD (Média F1): '{avg_kd_performance.idxmax()}' ({avg_kd_performance.max():.2f})")
    avg_prompt_performance = numeric_df.mean(axis=0)
    print(f"- Melhor Estratégia de Prompting (Média F1): '{avg_prompt_performance.idxmax()}' ({avg_prompt_performance.max():.2f})")
    best_combo = numeric_df.stack().idxmax()
    print(f"- Melhor Sinergia (Modelo + Prompt): '{best_combo[0]}' + '{best_combo[1]}' com F1 de {numeric_df.max().max():.2f}")

    print("\nGerando gráfico de comparação...")
    fig, ax = plt.subplots(figsize=(12, 7))
    numeric_df.plot(kind='bar', ax=ax, title='Comparação de Performance (F1-Score) entre Modelos e Estratégias')
    ax.set_ylabel("F1-Score")
    ax.set_xlabel("Versão do Modelo")
    ax.tick_params(axis='x', rotation=20)
    plt.legend(title='Estratégia de Prompt', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.savefig("real_run_full_performance_comparison.png")
    print("Gráfico salvo em 'real_run_full_performance_comparison.png'.")

In [ ]:
run_real_workflow(True, 10000)

--- FASE 0: CONFIGURANDO AMBIENTE E DADOS ---
--- Step 0.1: Configuring Task (SQuAD) and Metrics ---
Dataset e métricas carregados.

--- Step 0.4: Preparing Datasets ---
Datasets preparados.

--- FASE 1: EXECUTANDO TREINAMENTO DOS MODELOS KD ---

--- Loading Model: google/gemma-2b ---


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Modelo carregado com sucesso.

--- Loading Model: mistralai/Mistral-7B-Instruct-v0.2 ---


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Erro ao carregar o modelo: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

ERRO CRÍTICO: Falha ao carregar um dos modelos base. O pipeline não pode continuar.


In [ ]:
# --- FASE 0: SETUP ---
print("--- FASE 0: CONFIGURANDO AMBIENTE E DADOS ---")
validation_data, metrics_calc = configure_task_and_metrics()
prepared_data = prepare_datasets(validation_data)
config = KDConfig()

run_training  = True

--- FASE 0: CONFIGURANDO AMBIENTE E DADOS ---
--- Step 0.1: Configuring Task (SQuAD) and Metrics ---


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Dataset e métricas carregados.

--- Step 0.4: Preparing Datasets ---
Datasets preparados.


In [ ]:
# --- FASE 1: TREINAMENTO (Opcional) ---

print("\n--- FASE 1: EXECUTANDO TREINAMENTO DOS MODELOS KD ---")
# Carrega os modelos base necessários para todos os treinamentos
student_model_base, student_tokenizer = load_model("google/gemma-2b")
teacher_model_ext, teacher_tokenizer = load_model("mistralai/Mistral-7B-Instruct-v0.2")


--- FASE 1: EXECUTANDO TREINAMENTO DOS MODELOS KD ---

--- Loading Model: google/gemma-2b ---


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Modelo carregado com sucesso.

--- Loading Model: mistralai/Mistral-7B-Instruct-v0.2 ---


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Modelo carregado com sucesso.


In [ ]:
# 1. Treina o modelo de KD Base
run_base_kd_training(
    student_model=student_model_base, teacher_model=teacher_model_ext, student_tokenizer=student_tokenizer, teacher_tokenizer=teacher_tokenizer,
    kd_dataset=prepared_data['kd_transfer'], config=config, output_dir="./results/kd_base_model"
)

# 2. Treina o modelo de Auto-Destilação (requer um professor "especialista")
print("\n--- Treinando professor para Auto-Destilação ---")
# A forma mais simples de criar um professor especialista é com um fine-tuning padrão.
# Aqui, vamos simular isso treinando o próprio estudante com KD por uma época.
specialist_teacher = run_base_kd_training(
    student_model=student_model_base, teacher_model=teacher_model_ext, student_tokenizer=student_tokenizer, teacher_tokenizer=teacher_tokenizer,
    kd_dataset=prepared_data['kd_transfer'], config=config, output_dir="./results/specialist_teacher_for_self_distillation"
)
print("\n--- Executando Auto-Destilação ---")
run_base_kd_training(
    student_model=student_model_base, teacher_model=specialist_teacher, student_tokenizer=student_tokenizer, teacher_tokenizer=teacher_tokenizer,
    kd_dataset=prepared_data['kd_transfer'], config=config, output_dir="./results/kd_self_distilled_model"
)

# 3. Treina o modelo de Destilação de Explicações
explanation_dataset = generate_explanation_dataset(teacher_model_ext, teacher_tokenizer, prepared_data['kd_transfer'])
run_explanation_kd_training(
    student_model=student_model_base, student_tokenizer=student_tokenizer, teacher_tokenizer=teacher_tokenizer, explanation_dataset=explanation_dataset,
    config=config, output_dir="./results/explanation_distilled_model"
)
print("\n--- FASE DE TREINAMENTO CONCLUÍDA ---")


--- Running Base Knowledge Distillation -> saving to ./results/kd_base_model ---

Epoch 1/1


Epoch 1:   0%|          | 0/200 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 6.12 MiB is free. Process 367409 has 14.73 GiB memory in use. Of the allocated memory 14.58 GiB is allocated by PyTorch, and 30.64 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# --- FASE 2 & 3: AVALIAÇÃO E ANÁLISE ---
print("\n--- FASES 2 & 3: EXECUTANDO AVALIAÇÃO COMPLETA E GERANDO RELATÓRIO ---")
model_definitions = {
    "SLM Base": "google/gemma-2b",
    "KD Base": "./results/kd_base_model",
    "KD Auto-Destilado": "./results/kd_self_distilled_model",
    "KD com Explicações": "./results/explanation_distilled_model"
}
prompting_strategies = {
    "Zero-Shot Simples": run_zero_shot_baseline,
    "ICL (k=3)": lambda m, t, d, met: run_few_shot_icl(m, t, d, prepared_data['icl_cot_examples'], 3, met),
    "Zero-Shot CoT": run_zero_shot_cot,
    "Auto-Consistência (n=5)": lambda m, t, d, met: run_self_consistency_cot(m, t, d, met, 5)
}
results_df = pd.DataFrame(index=model_definitions.keys(), columns=prompting_strategies.keys())

In [ ]:
# Carrega o modelo base uma vez para aplicar os adaptadores PEFT
base_model_for_eval, tokenizer = load_model("google/gemma-2b")

for alias, path in model_definitions.items():
    print(f"\n>>>> AVALIANDO MODELO: {alias} <<<<")
    if alias == "SLM Base":
        model = base_model_for_eval
    else:
        try:
            # Carrega o modelo base com o adaptador LoRA treinado
            model = PeftModel.from_pretrained(base_model_for_eval, path)
            model = model.merge_and_unload() # Opcional: mescla os pesos para acelerar a inferência
        except Exception as e:
            print(f"Não foi possível carregar o modelo treinado de '{path}'. Pulando. Erro: {e}")
            continue

    for strat_name, strat_func in prompting_strategies.items():
        # A função de avaliação genérica agora é chamada por suas wrappers específicas
        results = strat_func(model, tokenizer, prepared_data['evaluation'], metrics_calc)
        results_df.loc[alias, strat_name] = f"{results['f1']:.1f} / {results['exact_match']:.1f}"

# --- RELATÓRIO FINAL ---
print("\n\n## Relatório Final de Análise ##")
numeric_df = results_df.applymap(lambda x: float(x.split('/')[0]) if isinstance(x, str) else 0)

print("\n--- Matriz de Resultados (F1-Score / Exact Match) ---")
print(results_df.to_markdown())

print("\n--- Conclusões da Análise ---")
avg_kd_performance = numeric_df.drop("SLM Base").mean(axis=1)
print(f"- Melhor Técnica de KD (Média F1): '{avg_kd_performance.idxmax()}' ({avg_kd_performance.max():.2f})")
avg_prompt_performance = numeric_df.mean(axis=0)
print(f"- Melhor Estratégia de Prompting (Média F1): '{avg_prompt_performance.idxmax()}' ({avg_prompt_performance.max():.2f})")
best_combo = numeric_df.stack().idxmax()
print(f"- Melhor Sinergia (Modelo + Prompt): '{best_combo[0]}' + '{best_combo[1]}' com F1 de {numeric_df.max().max():.2f}")

print("\nGerando gráfico de comparação...")
fig, ax = plt.subplots(figsize=(12, 7))
numeric_df.plot(kind='bar', ax=ax, title='Comparação de Performance (F1-Score) entre Modelos e Estratégias')
ax.set_ylabel("F1-Score")
ax.set_xlabel("Versão do Modelo")
ax.tick_params(axis='x', rotation=20)
plt.legend(title='Estratégia de Prompt', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig("real_run_full_performance_comparison.png")
print("Gráfico salvo em 'real_run_full_performance_comparison.png'.")